In [ ]:
%%bash
# preamble script to check and install AMUSE components if necessary

# required packages for this tutorial:
PACKAGES="amuse-framework"
# skip in case a full development install is present
pip show amuse-devel && exit 0
for package in ${PACKAGES} 
do
  pip show ${package} || pip install ${package}
done

In [ ]:
# the following fixes are highly recommended

#allow oversubscription for openMPI
import os
os.environ["OMPI_MCA_rmaps_base_oversubscribe"]="true"

# use lower cpu resources for idle codes
from amuse.support import options
options.GlobalOptions.instance().override_value_for_option("polling_interval_in_milliseconds", 10)


In [ ]:
%matplotlib inline
from matplotlib import pyplot
import numpy

AMUSE pre-defines a number of calculcated attributes on particle sets, such as the kinetic energy of the particles in the set. These calculated attributes are used often and provide a sufficient set to start out with, but they do not define a *complete* set. It's possible to define your own attributes and extend the attributes on a particle set.

In [ ]:
from amuse.lab import *

As shown in the previous example, you can create a particle set by specifying the number of particles and setting their attributes. You can also create a particle set by using an inital condition function. For stellar clusters the commonly used plummer and king models are available. For this tutorial we will start with a king model. Global clusters created with a king model need the number of stars in the cluster and a dimensionless depth parameter that determines the depth of the potential well in the center of the cluster.

In [ ]:
particles = new_king_model(1000, 3)
print(particles)

Common properties for a stellar cluster are its  center of mass position, total kinetic energy and potential energy.

In [ ]:
print("center of mass", particles.center_of_mass())
print("kinetic energy", particles.kinetic_energy())
print("potential energy", particles.potential_energy(G=nbody_system.G))

For the potential energy calculation we need to specify the gravitational constant, as the default value will use the gavitational constant in S.I. units and we are working in nbody units for this tutorial.

In N-body calculations and reporting, the kinetic and potential energy of a set of stars is often scaled to exactly 0.25 and -0.5 respectively. AMUSE also has a function for this.

In [ ]:
particles.scale_to_standard()
print("kinetic energy", particles.kinetic_energy())
print("potential energy", particles.potential_energy(G=nbody_system.G))

*Note that the potential energy and scaling calculations are implemented as order N-squared operations*

Attributes of particle sets are always 1 dimensional by default, an array with a single value per particle attribute. But for some attributes it is easier to work with a 2d set, an array with multiple values (or an array of values) per particle attribute. For example, the positions of all particles. These attributes are called vector-attributes and are defined as a combination of 2 or more simple attributes. 

The position attribute combines the values of the `x`, `y` and `z` attributes.

In [ ]:
print(particles[0].x)
print(particles[0].y)
print(particles[0].z)
print(particles[0].position)

Other common vector attributes are `velocity` (combination of `vx`,`vy`,`vz`) and `acceleration` (combination of `ax`,`ay`,`az`).

You can set the value of a position attribute and the underlying x, y or z attributes will be changed. 

In [ ]:
particles[0].position = [0, 0.1, 0.2] | nbody_system.length
print(particles[0].x)
print(particles[0].y)
print(particles[0].z)

You can set the value of the x, y or z attribute and the position will change (as the position is just a combination of these attributes).

In [ ]:
particles[0].x = 0.3 | nbody_system.length
print(particles[0].position)

You cannot change an item in the position array and thereby change the x, y, or z positions

In [ ]:
# this will not change anything in the particles set as the position is a copy
particles[0].position[0] = 0.5 | nbody_system.length

print(particles[0].x)
print(particles[0].position)

You can use the position attribute on the entire set. Let's print the positions of the first 10 particles.

In [ ]:
print(particles.position[0:10])

You can also use the position attribute to set values for the entire set

In [ ]:
# set the position of all particles in the set to the same value
particles.position = [0.1, 0.2, 0.3] | nbody_system.length

print(particles.position[0:10])
print(particles.x[0:10])

Defining a new vector attribute is done by calling the `add_vector_attribute` or `add_global_vector_attribute`. The first call will define the attribute on the particle set and not on any other set. The second call will define the attribute on the particle set and any future sets created in the script. (The second call is used in the amuse framework itself to define the `position`, `velocity` and `acceleration` attributes)

In [ ]:
particles.add_vector_attribute('position2d', ('x', 'y'))
print(particles[0].position2d)

If you enter `particles.add_` and press tab you'll notice two other function besides the `add_vector_attribute` function; `add_calculated_attribute` will create an attribute where the values are calculated based on other attributes, `add_function_attribute` will create a function on the set that gets the set and optional function parameters. These function also have global versions (`add_global_...`). The `add_global_function_attribute` call is used in the AMUSE framework to implement the `kinetic_energy` and `potential_energy` functions.


In [ ]:
particles.add_function_attribute(
    'calculate_mean_mass',
    lambda particles: particles.mass.sum() / len(particles)
)
print(particles.calculate_mean_mass())